In [ ]:
using NBInclude, NLopt
_PRECOMPILE_=true
_LC_=true
@nbinclude("startup_GER.ipynb")

# 1) Find the parameters of the benchmark model
We take as moments the expected duration of nonemployment and UI, and the cumulated duration of nonemployment.

We calibrate first our model to determine the values of $\alpha$, $m$ and $q_a$, $\lambda_1$ and $\lambda_0$, while fixing $q_0=0.02$.

In [ ]:
function distance_noq(m1,m2)
    out = 0
    for i in 1:length(m1[1])
        out += (m1[1][i] - m2[1][i]) ^2 / length(m1[1])
        out += (m1[2][i] - m2[2][i]) ^2 / length(m1[1])
    end
    return(out)
end



## compute the objective to minimise
function objective_ger_noq(α, m,q0, qa, λ1, λdiff)
    λ0=λ1+λdiff
    
    ## printed info
  #  IJulia.clear_output(true)
 #   print("Current parameters: $α, $m, $q0, $qa, $λ1, $λ0")
    
    ## simulate  
    newC = update_Country2(α=α, m=m, q0=q0, qa=qa, λ1=λ1, λ0=λ0)
    V = find_Surplus(newC,reltol=1e-12, findΨ=false) 
    moments_emp = moments_germany_UInoq(V,newC)
    out = distance_noq(moments_emp, moments_target_UI)
  #  print(" $(round(out,sigdigits=5))")
  #  println()
    return out
end

## distance minimisation with NLopt (/!\ time-consuming)
println()
println("Find α, m, λ1 and λ0 given q0=0.02.")
@time parmin_noq = let
    opt = Opt(:LN_BOBYQA, 6)
    lower_bounds!(opt, [0.1 , 0., 0.02, -0.001, 0., 0.])
    upper_bounds!(opt, [10. , 1., 0.02, 0.001, 0.2, 1.])
    xtol_rel!(opt,1e-12) 
    maxtime!(opt, 3600)
    min_objective!(opt, (v,g)-> objective_ger_noq(v[1],v[2],v[3],v[4],v[5],v[6]))
    
    init = [1., 0.1, 0.02, 0., 0.06, 0.03]

    (minf,minx,ret) = optimize(opt, init) 
    println("Parameters found with convergence $ret, the value of the objective is $minf")
    println("Parameters: ")
    println(" - α= ", minx[1])
    println(" - m= ", minx[2])
    println(" - q0= ", minx[3])
    println(" - qa= ", minx[4])
    println(" - λ1= ", minx[5])
    println(" - λ0= λ1 + ", minx[6])
    minx
end

We calibrate our model to determine the values of $\alpha$, $m$, $q_0$ and $q_A$, $\lambda_1$ and $\lambda_0$.

In [ ]:
function distance(m1,m2)
    out = 0
    for i in 1:length(m1[1])
        out += (m1[1][i] - m2[1][i]) ^2 / length(m1[1])
        out += (m1[2][i] - m2[2][i]) ^2 / length(m1[1])
    end
    out += (m1[3] - m2[3]) ^2
    return(out)
end

## compute the objective to minimise
function objective_ger(α, m,q0, qa, λ1, λdiff)
    λ0=λ1+λdiff
    
    ## printed info
  #  IJulia.clear_output(true)
  #  println("Current parameters: $α, $m, $q0, $qa, $λ1, $λ0")
    
    ## simulate  
    newC = update_Country2(α=α, m=m, q0=q0, qa=qa, λ1=λ1, λ0=λ0)
    V = find_Surplus(newC,reltol=1e-12, findΨ=false) 
    moments_emp = moments_germany_UI(V,newC)
    out = distance(moments_emp, moments_target_UI)
#    print(round(out,sigdigits=5))
    return out
end


function objective_ger2(α, m,q0, qa, λ1, λdiff)
    try objective_ger(α, m,q0, qa, λ1, λdiff)
    catch
        Inf
    end
end


## distance minimisation with NLopt (/!\ time-consuming)
@time parmin = let
    opt = Opt(:LN_SBPLX, 6)
    lower_bounds!(opt, [0.1 , 0.,0., -0.001, 0., 0.])
    upper_bounds!(opt, [10. , 1.,0.2, 0.001, 0.2, 1.])
    xtol_rel!(opt,1e-12) 
    maxtime!(opt, 47 * 3600)
    min_objective!(opt, (v,g)-> objective_ger2(v[1],v[2],v[3],v[4],v[5],v[6]))

    init = vcat(parmin_noq[1:5], sum(parmin_noq[5:6]))

    (minf,minx,ret) = optimize(opt, init) 
    println("Parameters found with convergence $ret, the value of the objective is $minf")
    println("Parameters: ")
    println(" - α= ", minx[1])
    println(" - m= ", minx[2])
    println(" - q0= ", minx[3])
    println(" - qa= ", minx[4])
    println(" - λ1= ", minx[5])
    println(" - λ0= λ1 + ", minx[6])
    minx
end
; 